In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [12]:
df = pd.read_excel("D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_train.xlsx")

In [13]:
df_truth = pd.read_excel("D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_truth.xlsx")

In [14]:
df_merged = pd.merge(df, df_truth, on='id')
df_merged

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,more
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,98
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,98
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,98
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,98
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20426,99,181,-0.0015,-0.0001,100,518.67,643.72,1600.39,1428.03,14.62,...,2388.29,8123.55,8.4885,0.03,396,2388,100,38.50,23.0425,20
20427,99,182,-0.0027,-0.0003,100,518.67,643.52,1605.33,1430.32,14.62,...,2388.27,8130.99,8.5124,0.03,393,2388,100,38.39,22.9674,20
20428,99,183,-0.0031,-0.0003,100,518.67,643.96,1606.95,1427.90,14.62,...,2388.31,8126.90,8.5374,0.03,395,2388,100,38.57,23.1440,20
20429,99,184,-0.0010,-0.0001,100,518.67,644.10,1600.20,1436.54,14.62,...,2388.33,8125.66,8.5592,0.03,395,2388,100,38.45,23.0478,20


In [15]:
# Step 1: Get the maximum cycle for each engine
max_cycle_per_engine = df.groupby('id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['id', 'max_cycle']

# Step 2: Merge the maximum cycle with the df_truth to get the actual failure cycle
df_merged = pd.merge(max_cycle_per_engine, df_truth, on='id')

# Step 3: Calculate the actual failure cycle (when engine will fail)
df_merged['failure_cycle'] = df_merged['max_cycle'] + df_merged['more']

# Step 4: Merge this back with the main DataFrame to compute remaining cycles
df = pd.merge(df, df_merged[['id', 'failure_cycle']], on='id')

# Step 5: Calculate remaining cycles for each row by subtracting the current cycle from the failure cycle
df['remaining_cycles'] = df['failure_cycle'] - df['cycle']
df = df.drop('failure_cycle',axis=1)

In [16]:
df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_cycles
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,289
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,288
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,287
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,286
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20426,99,181,-0.0015,-0.0001,100,518.67,643.72,1600.39,1428.03,14.62,...,2388.29,8123.55,8.4885,0.03,396,2388,100,38.50,23.0425,24
20427,99,182,-0.0027,-0.0003,100,518.67,643.52,1605.33,1430.32,14.62,...,2388.27,8130.99,8.5124,0.03,393,2388,100,38.39,22.9674,23
20428,99,183,-0.0031,-0.0003,100,518.67,643.96,1606.95,1427.90,14.62,...,2388.31,8126.90,8.5374,0.03,395,2388,100,38.57,23.1440,22
20429,99,184,-0.0010,-0.0001,100,518.67,644.10,1600.20,1436.54,14.62,...,2388.33,8125.66,8.5592,0.03,395,2388,100,38.45,23.0478,21


In [7]:
data= df.copy()

In [8]:
def create_sequences(data, sequence_length, downsample=1):
    """
    Creates sequences of given length, with optional downsampling.
    
    Parameters:
    - data: DataFrame containing the data
    - sequence_length: The length of the sequences
    - downsample: Factor by which to downsample within the sequence
    
    Returns:
    - List of sequences (each sequence is an array of shape [sequence_length, features])
    - List of corresponding remaining_cycles (targets for each sequence)
    """
    sequences = []
    targets = []

    # Group by 'id' to handle each engine separately
    for engine_id in data['id'].unique():
        engine_data = data[data['id'] == engine_id]

        # Loop through each cycle to create sequences
        for i in range(len(engine_data) - sequence_length + 1):
            # Extract the sequence and downsample if required
            sequence = engine_data.iloc[i:i + sequence_length].iloc[::downsample]
            target = engine_data['remaining_cycles'].iloc[i + sequence_length - 1]
            
            sequences.append(sequence.drop(columns=['id', 'cycle', 'remaining_cycles']).values)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

# Short-term sequences (no downsampling)
short_term_sequences, short_term_targets = create_sequences(data, sequence_length=10, downsample=1)

# Medium-term sequences (downsample by a factor of 5 to reduce length)
medium_term_sequences, medium_term_targets = create_sequences(data, sequence_length=50, downsample=5)

# Check shapes
print("Short-term sequences shape:", short_term_sequences.shape)
print("Short-term targets shape:", short_term_targets.shape)
print("Medium-term sequences shape:", medium_term_sequences.shape)
print("Medium-term targets shape:", medium_term_targets.shape)

Short-term sequences shape: (19540, 10, 24)
Short-term targets shape: (19540,)
Medium-term sequences shape: (15580, 10, 24)
Medium-term targets shape: (15580,)


In [9]:
import pandas as pd
import numpy as np

# Define function to create sequences for different scales
def create_aligned_sequences(data, short_term_len, medium_term_len, downsample=1):
    short_term_sequences = []
    medium_term_sequences = []
    targets = []

    for engine_id in data['id'].unique():
        engine_data = data[data['id'] == engine_id]

        # Minimum range to allow both short- and medium-term sequences to fit
        max_start = len(engine_data) - max(short_term_len, medium_term_len) + 1

        for i in range(max_start):
            # Short-term sequence (no downsampling)
            short_sequence = engine_data.iloc[i:i + short_term_len]
            # Medium-term sequence with downsampling
            medium_sequence = engine_data.iloc[i:i + medium_term_len].iloc[::downsample]

            # Ensure the lengths are as expected
            if len(short_sequence) == short_term_len and len(medium_sequence) == (medium_term_len // downsample):
                short_term_sequences.append(short_sequence.drop(columns=['id', 'cycle', 'remaining_cycles']).values)
                medium_term_sequences.append(medium_sequence.drop(columns=['id', 'cycle', 'remaining_cycles']).values)
                # Target at the end of the sequence
                targets.append(engine_data['remaining_cycles'].iloc[i + short_term_len - 1])

    return np.array(short_term_sequences), np.array(medium_term_sequences), np.array(targets)

# Generate aligned sequences
short_term_sequences, medium_term_sequences, targets = create_aligned_sequences(
    data, short_term_len=10, medium_term_len=50, downsample=5
)


# Check shapes
print("Short-term sequences shape:", short_term_sequences.shape)
print("Short-term targets shape:", short_term_targets.shape)
print("Medium-term sequences shape:", medium_term_sequences.shape)
print("Medium-term targets shape:", medium_term_targets.shape)


Short-term sequences shape: (15580, 10, 24)
Short-term targets shape: (19540,)
Medium-term sequences shape: (15580, 10, 24)
Medium-term targets shape: (15580,)


In [10]:
# Define separate models for short-term and medium-term predictions
def create_short_term_model(input_shape):
    input_layer = Input(shape=input_shape)
    lstm_layer = LSTM(128, return_sequences=True)(input_layer)
    lstm_layer = Dropout(0.3)(lstm_layer)
    lstm_layer = LSTM(64)(lstm_layer)
    output_layer = Dense(1)(lstm_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

def create_medium_term_model(input_shape):
    input_layer = Input(shape=input_shape)
    lstm_layer = LSTM(128, return_sequences=True)(input_layer)
    lstm_layer = Dropout(0.3)(lstm_layer)
    lstm_layer = LSTM(64)(lstm_layer)
    output_layer = Dense(1)(lstm_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Create and compile the models
short_term_model = create_short_term_model(short_term_sequences.shape[1:])
medium_term_model = create_medium_term_model(medium_term_sequences.shape[1:])

short_term_model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
medium_term_model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# Compile the model with a lower learning rate and MAE loss
initial_learning_rate = 0.00001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)


# Implement learning rate schedule
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: initial_learning_rate * (0.1 ** (epoch // 10))
)

: 

In [ ]:
# Train each model separately
short_term_history = short_term_model.fit(
    short_term_sequences,
    short_term_targets,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[lr_schedule]
)

medium_term_history = medium_term_model.fit(
    medium_term_sequences,
    medium_term_targets,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[lr_schedule]
)

Epoch 1/100
195/195 [==============================] - 8s 22ms/step - loss: 172.5944 - mae: 172.5944 - val_loss: 180.2815 - val_mae: 180.2815 - lr: 1.0000e-05
Epoch 2/100
195/195 [==============================] - 3s 17ms/step - loss: 172.0046 - mae: 172.0046 - val_loss: 179.5676 - val_mae: 179.5676 - lr: 1.0000e-05
Epoch 3/100
195/195 [==============================] - 3s 16ms/step - loss: 171.2340 - mae: 171.2340 - val_loss: 178.6143 - val_mae: 178.6143 - lr: 1.0000e-05
Epoch 4/100
195/195 [==============================] - 3s 18ms/step - loss: 170.2404 - mae: 170.2404 - val_loss: 177.5325 - val_mae: 177.5325 - lr: 1.0000e-05
Epoch 5/100
195/195 [==============================] - 3s 17ms/step - loss: 169.1969 - mae: 169.1969 - val_loss: 176.5832 - val_mae: 176.5832 - lr: 1.0000e-05
Epoch 6/100
195/195 [==============================] - 3s 17ms/step - loss: 168.3382 - mae: 168.3382 - val_loss: 175.8645 - val_mae: 175.8645 - lr: 1.0000e-05
Epoch 7/100
195/195 [=========================

In [ ]:
from sklearn.model_selection import train_test_split

# Get unique engine IDs
engine_ids = data['id'].unique()

# Split engine IDs into training and testing sets (e.g., 80% train, 20% test)
train_ids, test_ids = train_test_split(engine_ids, test_size=0.2, random_state=42)

# Filter data for train and test sets
train_data = data[data['id'].isin(train_ids)]
test_data = data[data['id'].isin(test_ids)]

In [ ]:
# Generate sequences for training
short_term_train, medium_term_train, train_targets = create_aligned_sequences(
    train_data, short_term_len=10, medium_term_len=50, downsample=5
)

# Generate sequences for testing
short_term_test, medium_term_test, test_targets = create_aligned_sequences(
    test_data, short_term_len=10, medium_term_len=50, downsample=5
)

In [ ]:
# Make predictions with both models
short_term_predictions = short_term_model.predict(short_term_test_sequences)
medium_term_predictions = medium_term_model.predict(medium_term_test_sequences)

# Average the predictions
final_predictions = (short_term_predictions.flatten() + medium_term_predictions.flatten()) / 2

# Compare predictions with actual values
for i in range(10):
    print(f"Predicted: {final_predictions[i]:.2f}, Actual: {actual_rul[i]}")